# Part One: Getting the list of all Properties

First we will set up a script to download all the most up-to-date tax information on properties in Syracuse

In [2]:
# Import Library
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
# Open Browser
driver = webdriver.Chrome(executable_path='C:/Users/humme/Downloads/chromedriver.exe')
# Get the  URL
url = 'https://ocfintax.ongov.net/Imate/index.aspx'
driver.get(url)

In [ ]:
element = driver.find_element(By.ID,value='btnPublicAccess')
element.click();

element = driver.find_element(By.ID,value='chkAgree')
element.click();
time.sleep(0.25) #sleep for 250 milliseconds
element = driver.find_element(By.ID,value='btnSubmit')
element.click();

element = driver.find_element(By.ID,value='btnSearch')
element.click();

In [ ]:
pages = int(driver.find_element(By.ID,value="lblPageCount").text)
converters = {"Property #":str}
Tax_Database = pd.read_html(driver.find_element(by=By.XPATH, value='//*[@id="tblList"]').get_attribute('outerHTML'), header = 0, converters = converters)[0]

In [ ]:
for i in range(pages-1):
        element = driver.find_element(By.ID,value='lnkNextPage')
        element.click()
        new = pd.read_html(driver.find_element(by=By.XPATH, value='//*[@id="tblList"]').get_attribute('outerHTML'), header = 0, converters = converters)[0]
        Tax_Database = pd.concat([Tax_Database, new])
        print(len(Tax_Database))
        time.sleep(0.25) #sleep for 250 milliseconds

In [ ]:
driver.close()
driver.quit()

In [ ]:
import os
from pandas import ExcelWriter
os.chdir("C:/Users/"+ str(os.environ['USERNAME'])+"/Downloads/")
with ExcelWriter('syracuse_house_data.xlsx', date_format='MM/DD/YYYY', datetime_format='MM/DD/YYYY') as writer:
    Tax_Database.to_excel(writer, index=False)

# Part One A:
We will clean the data and match it with the data from foil

In [ ]:
foil = pd.read_excel("C:/Users/humme/My Drive/CUNY Classes/608 - Knowledge and Visual Analytics/Syracuse_foil_data_2022.xlsx")

In [ ]:
collect = pd.read_excel("C:/Users/humme/Downloads/syracuse_house_data.xlsx", converters = {"Property #":str})

Now, let's rename and clean some of these columns as well as check to see what the data looks like.

In [ ]:
foil.columns = ['Key Map', 'Index', 'Prop Address', 'Owner Name', 'Org Tax']

In [ ]:
foil['Prop Address'] = foil['Prop Address'].str.lower()

In [ ]:
foil.head()

In [ ]:
collect.columns = ['Prop Number', 'Municipality', 'Tax ID', 'Owner', 'Street Number', "Street Name"]

In [ ]:
collect.head()

It looks like the Key Map Column is the Municipality number as well as the tax id combined together. So we will create a new column in collect that includes that so we can merge them. 

In [ ]:
collect['key'] = collect['Municipality'].str.extract(r'(\d*([^-]+))')[0]

In [ ]:
collect['Key Map'] = collect['key'] + collect['Tax ID']

In [ ]:
collect['Key Map'] = collect['Key Map'].str.replace(' ', '')

In [ ]:
collect.head()

Now that we have the primary key, we can merge the data. And we will check the merge.

In [ ]:
combined = collect.merge(foil, how='left', on='Key Map')

In [ ]:
combined.head()

In [ ]:
combined.describe()

In [ ]:
combined.isnull( ).sum()

In [ ]:
combined.nunique()

FOIL Told me that they can only give me data for the suburbs and not the city. So we need to check to see if all of the NA Properties for Prop Adress are City Properties. We can do this by seeing if there is an equal number of 'Prop Numbers' as there are NAs. Prop Number was a column unique to city adresses that will also be used to get data online for the tax amount. 

In [ ]:
combined_check = combined.loc[combined['Index'].isnull()] 

In [ ]:
combined_check.isnull( ).sum()

So we have 3k that do no have a prop number to get the information online. First, let's see if there are any city addresses that are blank. 

In [ ]:
combined_check['Municipality'].loc[combined_check['Prop Number'].isnull()].value_counts()

It does not look like that there aren't any city addresses, only suburbs. When I looked up a few of the houses, I found that they were new construction and still on the market having never paid taxes. We can check this by seeing the more popular streets where the houses are on.

In [ ]:
l = pd.DataFrame(combined_check.loc[combined_check['Prop Number'].isnull()].groupby(['Municipality','Street Name']).size().to_frame(name = 'count').reset_index()).sort_values(by = ['Municipality','count'], ascending = False)

In [ ]:
l.loc[l['count'] > 5]

It does look like that most are new construction! So we will exclude these from the data set. 

In [ ]:
remove_list  = list(combined_check['Key Map'].loc[combined_check['Prop Number'].isnull()])

In [ ]:
len_rem = len(remove_list)
len_comb = len(combined)

In [ ]:
combined = combined[~combined['Key Map'].isin(remove_list)]

In [ ]:
len_comb - len_rem == len(combined)

Now another issue I noticed is that there might be 2+ people registered to an address that combined will pay taxes. For example, my wife Shwetha and I.

In [ ]:
combined.loc[(combined['Street Name'].str.contains("Clarendon")) & (combined['Street Number'].str.contains("515"))]

In [ ]:
combined = combined.drop_duplicates(subset=['Key Map'], keep='first', inplace=False, ignore_index=False).reset_index()

Now we will save this dataset and move on to the next step

In [ ]:
import os
from pandas import ExcelWriter
os.chdir("C:/Users/"+ str(os.environ['USERNAME'])+"/Downloads/")
with ExcelWriter('syracuse_house_data_combined.xlsx', date_format='MM/DD/YYYY', datetime_format='MM/DD/YYYY') as writer:
    combined.to_excel(writer, index=False)

# Part Two: Getting the Tax Dollars for City Taxes

As mentioned above, there are city houses that do not have county/town taxes so we have to get them and add them to the data

In [ ]:
new = pd.read_excel("C:/Users/humme/Downloads/syracuse_house_data_combined.xlsx", converters = {"Prop Number":str})

In [ ]:
s_prop = new.loc[~new['Prop Number'].isnull()]

In [ ]:
s_prop.head()

In [ ]:
s_list = list(set(s_prop['Prop Number']))

In [ ]:
tot = len(set(s_list))

I would not recommend running the following code because it took 5+ hours to run (I ran it while asleep).

In [ ]:
# Open Browser
tax_amount = []
count = 0
start1 = time.time()

driver = webdriver.Chrome(executable_path='C:/Users/humme/Downloads/chromedriver.exe')

for i in s_list:
    start = time.time()
    url = "https://syracuse.go2gov.net/faces/accounts?number=" + str(i) + "&src=SDG"
    driver.get(url)
    count+=1
   
    
    if "etaxTemplateForm:current1:0:seq1" in driver.page_source:
        element = driver.find_element(By.ID,value='etaxTemplateForm:current1:0:seq1')
        element.click();
        
        Tax_Database = pd.read_html(driver.find_element(by=By.XPATH, value='//*[@id="etaxTemplateForm:entities1"]').get_attribute('outerHTML'), header = 0)[0]
        city = Tax_Database['Tax Amount'].loc[Tax_Database['Taxing Purpose']=='CITY'][0]
        driver.back()
        
        Tax_Database = pd.read_html(driver.find_element(by=By.XPATH, value='//*[@id="etaxTemplateForm:current1"]').get_attribute('outerHTML'), header = 0)[0]
        county = Tax_Database['Tax Amount'].loc[Tax_Database['Unnamed: 0'].str.contains('County')][1]
        tax_amount.append({
            'Prod Number': i,
            'City_Tax': city,
            'County_Tax':county
        })
    else:
        city = 'NA'
        county = 'NA'
        tax_amount.append({
            'Prod Number': i,
            'City_Tax': city,
            'County_Tax':county
        })
    print(count," | ", tot)

pd.DataFrame(tax_amount)

In [ ]:
taxes = pd.DataFrame(tax_amount)

In [ ]:
u_cols = ['City_Tax','County_Tax']
for i in u_cols:  
    if ',' in  taxes[i]:
        taxes[i] = taxes[i].str.replace(',','')
        taxes[i] = taxes[i].str.replace('$','')
    else:
        taxes[i] = taxes[i].str.replace('$','')
    
for i in u_cols:
    taxes[i] = pd.to_numeric(taxes[i], errors='coerce')

In [ ]:
import os
from pandas import ExcelWriter
os.chdir("C:/Users/"+ str(os.environ['USERNAME'])+"/Downloads/")
with ExcelWriter('syracuse_house_tax.xlsx', date_format='MM/DD/YYYY', datetime_format='MM/DD/YYYY') as writer:
    taxes.to_excel(writer, index=False)

# Part 3 More Cleaning and data gathering
First, we need to download the data. I have already recieved access through a FOIL request or scraped the data online and the data we got from NYS.

I had to also get data from NYS so that we can match it up and get the building classifications.

In [3]:
## Asses is what we got from NYS
asses = pd.read_table("C:/Users/humme/Downloads/Prop_Assess.csv", 
                      encoding='utf-8', 
                      header=0, 
                      sep = ",", 
                      dtype = {'School District Code':object, 'Municipality Code':object},
                      usecols = ['Municipality Code','Municipality Name','School District Code','School District Name','SWIS Code','Property Class','Property Class Description','Print Key Code','Parcel Address Number','Parcel Address Street','Parcel Address Suff','Primary Owner First Name','Primary Owner Last Name','Full Market Value','Assessment Land','Assessment Total','County Taxable Value','Town Taxable Value','School Taxable']
)
## House Tax is the file we just got from getting all the county - town taxes and prop number
tax = pd.read_excel("C:/Users/humme/Downloads/syracuse_house_tax.xlsx", 
                    converters = {"Prod Number":str})

## Data combined is from part one and includes the data we originally scraped as well as the foil data
total = pd.read_excel("C:/Users/humme/Downloads/syracuse_house_data_combined.xlsx", 
                      converters = {"Prop Number":str})

In [4]:
tax = tax.rename(columns={'Prod Number': 'Prop Number'})

I want to merge the data file that I have for City Taxes (something I had to create myself) and the out of city numbers that were given in the foil request.

In [5]:
total = total.merge(tax, how='left', on='Prop Number')

now I want to add up the city and county taxes to get the org number that the total has

In [6]:
total['Org Tax'].loc[~total['Prop Number'].isna()] = total['City_Tax'].fillna(0).loc[~total['Prop Number'].isna()] + total['County_Tax'].fillna(0).loc[~total['Prop Number'].isna()]

C:\Users\humme\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
set(total['Org Tax'].loc[~total['Prop Number'].isna()] == total['City_Tax'].fillna(0).loc[~total['Prop Number'].isna()] + total['County_Tax'].fillna(0).loc[~total['Prop Number'].isna()])

{True}

With the QA Above, looks good. Now I will split the org into town and city for the foil request. They informed me that roughly the split is 55% to county and 45% to town/city

In [9]:
total['City_Tax'].loc[total['Prop Number'].isna()] = round(total['Org Tax'].loc[total['Prop Number'].isna()] * .45,2)
total['County_Tax'].loc[total['Prop Number'].isna()] = round(total['Org Tax'].loc[total['Prop Number'].isna()] * .55,2)

C:\Users\humme\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
set(total['Municipality'])

{'311500 - City of Syracuse',
 '312001 - Village of Camillus',
 '312089 - Town of Camillus',
 '312201 - Village of North Syracuse (Town of Cicero)',
 '312289 - Town of Cicero',
 '312401 - Village of North Syracuse (Town of Clay)',
 '312489 - Town of Clay',
 '312601 - Village of East Syracuse (Town of Dewitt)',
 '312689 - Town of Dewitt',
 '312801 - Village of Elbridge (Town of Elbridge)',
 '312803 - Village of Jordan (Town of Elbridge)',
 '312889 - Town of Elbridge',
 '313001 - Village of Fabius',
 '313089 - Town of Fabius',
 '313201 - Village of Solvay (Town of Geddes)',
 '313289 - Town of Geddes',
 '313400 - Town of Lafayette',
 '313601 - Village of Baldwinsville (Town of Lysander)',
 '313689 - Town of Lysander',
 '313801 - Village of Fayetteville (Town of Manlius)',
 '313803 - Village of Manlius (Town of Manlius)',
 '313805 - Village of Minoa (Town of Manlius)',
 '313889 - Town of Manlius',
 '314001 - Village of Marcellus',
 '314089 - Town of Marcellus',
 '314200 - Town of Onondaga'

In [11]:
total['Municipality Code'] = total['Municipality']

In [12]:
total['Municipality Code'] = total['Municipality Code'].replace({'311500 - City of Syracuse':'311500',
 '312001 - Village of Camillus':'312000',
 '312089 - Town of Camillus':'312000',
 '312201 - Village of North Syracuse (Town of Cicero)':'312200',
 '312289 - Town of Cicero':'312200',
 '312401 - Village of North Syracuse (Town of Clay)':'312400',
 '312489 - Town of Clay':'312400',
 '312601 - Village of East Syracuse (Town of Dewitt)':'312600',
 '312689 - Town of Dewitt':'312600',
 '312801 - Village of Elbridge (Town of Elbridge)':'312800',
 '312803 - Village of Jordan (Town of Elbridge)':'312800',
 '312889 - Town of Elbridge':'312800',
 '313001 - Village of Fabius':'313000',
 '313089 - Town of Fabius':'313000',
 '313201 - Village of Solvay (Town of Geddes)':'313200',
 '313289 - Town of Geddes':'313200',
 '313400 - Town of Lafayette':'313400',
 '313601 - Village of Baldwinsville (Town of Lysander)':'313600',
 '313689 - Town of Lysander':'313600',
 '313801 - Village of Fayetteville (Town of Manlius)':'313800',
 '313803 - Village of Manlius (Town of Manlius)':'313800',
 '313805 - Village of Minoa (Town of Manlius)':'313800',
 '313889 - Town of Manlius':'313800',
 '314001 - Village of Marcellus':'314000',
 '314089 - Town of Marcellus':'314000',
 '314200 - Town of Onondaga':'314200',
 '314400 - Town of Otisco':'314400',
 '314600 - Town of Pompey':'314600',
 '314801 - Village of Liverpool (Town of Salina)':'314800',
 '314889 - Town of Salina':'314800',
 '315001 - Village of Skaneateles':'315000',
 '315089 - Town of Skaneateles':'315000',
 '315200 - Town of Spafford':'315200',
 '315401 - Village of Tully':'315200',
 '315489 - Town of Tully':'315400',
 '315601 - Village of Baldwinsville (Town of Van Buren)':'315400',
 '315689 - Town of Van Buren':'315600'
 })

Now that we have all the columns we need, I'll merge it with the state data. The reason I got the onondaga county data from online is that the state data still has not been updated. I.E the house I own is still in the former owner's name. So we will match for the assement and property type using Key Map, which is the School Code and the SWIS Code.

In [13]:
asses.head(2)

,Municipality Code,Municipality Name,School District Code,School District Name,SWIS Code,Property Class,Property Class Description,Print Key Code,Parcel Address Number,Parcel Address Street,Parcel Address Suff,Primary Owner First Name,Primary Owner Last Name,Full Market Value,Assessment Land,Assessment Total,County Taxable Value,Town Taxable Value,School Taxable
0,311500,Syracuse,311500,Syracuse,311500,474.0,Billboards,001.1-01-03.0,635,Seventh North St & Bear T,NaN,NaN,Cooper Crouse-Hinds LLC,330738,242100,246400,246400,246400,246400
1,311500,Syracuse,311500,Syracuse,311500,449.0,"Other Storage, Warehouse and Distribution Faci...",001.1-01-04.0,1025-57,Hiawatha Blvd E & Seventh,NaN,NaN,Mic-Ran LLC,2416107,674900,1800000,1800000,1800000,1800000


In [14]:
asses['Key Map'] = asses['Municipality Code'] + asses['Print Key Code']

In [15]:
total['Key Map'] = total['Municipality Code'] + total['Tax ID']

In [16]:
total = total.merge(asses[['Key Map','Municipality Name','Property Class','Property Class Description','Full Market Value', 'Assessment Land', 'Assessment Total']], how='left', on='Key Map')

In [17]:
total.isnull().sum()

index                              0
Prop Number                   141396
Municipality                       0
Tax ID                             0
Owner                              1
Street Number                  14881
Street Name                      446
key                                0
Key Map                            0
Index                          41561
Prop Address                   41561
Owner Name                     41567
Org Tax                            0
City_Tax                        7880
County_Tax                     29664
Municipality Code                  0
Municipality Name               1308
Property Class                  1308
Property Class Description      1308
Full Market Value               1308
Assessment Land                 1308
Assessment Total                1308
dtype: int64

In [18]:
errors = total.loc[total['Municipality Name'].isna()]

In [19]:
total = total.loc[~total['Municipality Name'].isna()]

In [20]:
total.isnull().sum()

index                              0
Prop Number                   140176
Municipality                       0
Tax ID                             0
Owner                              1
Street Number                  14708
Street Name                      443
key                                0
Key Map                            0
Index                          41473
Prop Address                   41473
Owner Name                     41479
Org Tax                            0
City_Tax                        7792
County_Tax                     29576
Municipality Code                  0
Municipality Name                  0
Property Class                     0
Property Class Description         0
Full Market Value                  0
Assessment Land                    0
Assessment Total                   0
dtype: int64

When digging into some of the reasons that there were errors, the biggest is that the map key was different between nys and Syracuses data, but that the addresses were similar. So I will take advantage of Fuzzy Matcher to match up tax ids, street numberm and street name to fill in the NAs

In [21]:
import fuzzymatcher
left_on = ['Tax ID', 'Street Number','Street Name']
right_on = ['Print Key Code', 'Parcel Address Number','Parcel Address Street']

match = fuzzymatcher.fuzzy_left_join(errors, 
                             asses,
                             left_on,
                             right_on)

In [22]:
match.sort_values(by='best_match_score', ascending=False).head(3)

,best_match_score,__id_left,__id_right,index,Prop Number,Municipality,Tax ID,Owner,Street Number,Street Name,...,Parcel Address Suff,Primary Owner First Name,Primary Owner Last Name,Full Market Value_right,Assessment Land_right,Assessment Total_right,County Taxable Value,Town Taxable Value,School Taxable,Key Map_right
139772,1.225810,1299_left,178415_right,277383,NaN,315601 - Village of Baldwinsville (Town of Van...,656.001-9999-132.350/2883,Niagara Mohawk dba Nat'l Grid,NaN,Gas Distribution,...,NaN,NaN,Niagara Mohawk dba Nat'l Grid,185626,0,185626,185626,185626,185626,315600656.001-9999-132.350/2883
139769,1.212408,1296_left,178412_right,277377,NaN,315601 - Village of Baldwinsville (Town of Van...,656.001-9999-132.350/1013,Niagara Mohawk dba Nat'l Grid,NaN,Electric Transmission,...,NaN,NaN,Niagara Mohawk dba Nat'l Grid,14339,0,14339,14339,14339,14339,315600656.001-9999-132.350/1013
139770,1.206538,1297_left,178413_right,277379,NaN,315601 - Village of Baldwinsville (Town of Van...,656.001-9999-132.350/1023,Niagara Mohawk dba Nat'l Grid,NaN,Electric Transmission,...,NaN,NaN,Niagara Mohawk dba Nat'l Grid,14099,0,14099,14099,14099,14099,315600656.001-9999-132.350/1023


In [23]:
match.columns = match.columns.str.replace('_right', '')

In [24]:
match_clean = match[['index', 'Prop Number', 'Municipality', 'Tax ID', 'Owner',
       'Street Number', 'Street Name', 'key', 'Key Map', 'Index',
       'Prop Address', 'Owner Name', 'Org Tax', 'City_Tax', 'County_Tax',
       'Municipality Code', 'Municipality Name', 'Property Class',
       'Property Class Description', 'Full Market Value', 'Assessment Land',
       'Assessment Total']]

In [25]:
total = total.append([match_clean], ignore_index=True, sort=False)

In [26]:
total.isnull().sum()

index                              0
Prop Number                   141396
Municipality                       0
Tax ID                             0
Owner                              1
Street Number                  14881
Street Name                      446
key                                0
Key Map                            0
Index                          41561
Prop Address                   41561
Owner Name                     41567
Org Tax                            0
City_Tax                        7880
County_Tax                     29664
Municipality Code                  0
Municipality Name                  0
Property Class                     0
Property Class Description         0
Full Market Value                  0
Assessment Land                    0
Assessment Total                   0
dtype: int64

In [27]:
import os
from pandas import ExcelWriter
os.chdir("C:/Users/"+ str(os.environ['USERNAME'])+"/Downloads/")
with ExcelWriter('Syracuse_Homes_Final_Clean_Data.xlsx', date_format='MM/DD/YYYY', datetime_format='MM/DD/YYYY') as writer:
    total.to_excel(writer, index=False)